In [28]:
 # All imports
import pandas as pd 
import os
import json
import torch
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [17]:
data = "/kaggle/input/llm-classification-finetuning"

# getting file paths from the directory
file_paths = {"train.csv": "", "test.csv": ""}
for dirname, _, filenames in os.walk(data):
    for filename in filenames:
        fullpath = os.path.join(dirname, filename)
        # print(fullpath)
        if filename in file_paths:
            file_paths[filename] = fullpath

In [18]:
file_paths

{'train.csv': '/kaggle/input/llm-classification-finetuning/train.csv',
 'test.csv': '/kaggle/input/llm-classification-finetuning/test.csv'}

In [20]:
# reading data from file
df = pd.read_csv(file_paths['train.csv'])

# converting key columns into strings, joining all the items in the list & removing None in list
df["prompt"] = df["prompt"].apply(lambda x: ' '.join(json.loads(x)))
df["response_a"] = df["response_a"].apply(lambda x: ' '.join([i for i in json.loads(x) if i is not None]))
df["response_b"] = df["response_b"].apply(lambda x: ' '.join([i for i in json.loads(x) if i is not None]))

df.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I don't have personal beliefs or opi...",1,0,0
1,53567,koala-13b,gpt-4-0613,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...,0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,explain function calling. how would you call a...,Function calling is the process of invoking or...,Function calling is the process of invoking a ...,0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,How can I create a test set for a very rare ca...,Creating a test set for a very rare category c...,When building a classifier for a very rare cat...,1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,What is the best way to travel from Tel-Aviv t...,The best way to travel from Tel Aviv to Jerusa...,The best way to travel from Tel-Aviv to Jerusa...,0,1,0


In [22]:
# create labels
label = []
for index, row in df.iterrows():
    if row["winner_model_a"] == 1:
        label.append(0)
    elif row["winner_model_b"] == 1:
        label.append(1)
    else:
        label.append(2) 


df["label"] = label

df.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,label
0,30192,gpt-4-1106-preview,gpt-4-0613,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I don't have personal beliefs or opi...",1,0,0,0
1,53567,koala-13b,gpt-4-0613,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...,0,1,0,1
2,65089,gpt-3.5-turbo-0613,mistral-medium,explain function calling. how would you call a...,Function calling is the process of invoking or...,Function calling is the process of invoking a ...,0,0,1,2
3,96401,llama-2-13b-chat,mistral-7b-instruct,How can I create a test set for a very rare ca...,Creating a test set for a very rare category c...,When building a classifier for a very rare cat...,1,0,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,What is the best way to travel from Tel-Aviv t...,The best way to travel from Tel Aviv to Jerusa...,The best way to travel from Tel-Aviv to Jerusa...,0,1,0,1


In [1]:
# Adding prompts to the responses columns
df["response_a"] = "Prompt: " + df["prompt"] + " Response: " + df["response_a"]
df["response_b"] = "Prompt: " + df["prompt"] + " Response: " + df["response_b"]

model_in = []

for index, row in df.iterrows():
    model_in.append(["Prompt: " + row["prompt"]  + "Response: " + row["response_a"], 
     "Prompt: " + row["prompt"] + "Response: " + row["response_b"]])
    
df["model_in"] 

SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' on line 8 (3437517584.py, line 9)

In [ ]:
# splitting data in training and validation set
train_df, valid_df = train_test_split(df, test_size=0.2, stratify=df["label"])

In [29]:
model_checkpoint = "microsoft/deberta-v3-small"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

2025-08-20 20:18:27.617125: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755721107.972274      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755721108.071064      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin:   0%|          | 0.00/286M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/286M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
